In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [ ]:
## scrap the data from the website url
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.adamsstreetpartners.com/insights/the-next-frontier-the-rise-of-agentic-ai/")
loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
docs=loader.load()

In [5]:
##text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=70)
chunks = splitter.split_documents(docs)

In [6]:
chunks

[Document(metadata={'source': 'https://www.adamsstreetpartners.com/insights/the-next-frontier-the-rise-of-agentic-ai/', 'title': 'The Next Frontier: The Rise of Agentic AI', 'description': 'Agentic AI systems hold the potential to achieve specific goals by dynamically interacting with human and model inputs to make decisions autonomously.', 'language': 'en-US'}, page_content='The Next Frontier: The Rise of Agentic AI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to Main Content\n\n\nClose Search Box\n\n\n\nSearch for:\n\n\n\n\n Search'),
 Document(metadata={'source': 'https://www.adamsstreetpartners.com/insights/the-next-frontier-the-rise-of-agentic-ai/', 'title': 'The Next Frontier: The Rise of Agentic AI', 'description': 'Agentic AI systems hold the potential to achieve specific goals by dynamically interacting with human and model inputs to make 

In [7]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [9]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(chunks, embeddings)

In [ ]:
##querying from a vectorstore database
query = "Improved reasoning, planning, and multistep execution enable greater autonomy."
result = db.similarity_search(query)

In [16]:
result[0].page_content

'Advances in Large Language Models: Improved reasoning, planning, and multistep execution enable greater autonomy.\nMemory and Context Windows: Enhanced memory architectures allow AI agents to retain long-term context, improving performance on evolving tasks.\nReinforcement Learning and Fine Tuning: AI agents can self-improve through reinforcement learning and environment interactions.'

In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    '''
Answer the following question based on the provided context:
<context>
{context}
</context>
    '''
    )

document_chain = create_stuff_documents_chain(llm,prompt)

In [22]:
#using Retriever

retriever = db.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever,document_chain)

In [23]:
response = retriever_chain.invoke({"input": "While many agentic AI applications are in early development, potential use cases are emerging across industries and functions."})

In [24]:
response

{'input': 'While many agentic AI applications are in early development, potential use cases are emerging across industries and functions.',
 'context': [Document(id='06915818-815f-4150-b64d-2fddb9da0d94', metadata={'source': 'https://www.adamsstreetpartners.com/insights/the-next-frontier-the-rise-of-agentic-ai/', 'title': 'The Next Frontier: The Rise of Agentic AI', 'description': 'Agentic AI systems hold the potential to achieve specific goals by dynamically interacting with human and model inputs to make decisions autonomously.', 'language': 'en-US'}, page_content='What are the Potential Use Cases?\nWhile many agentic AI applications are in early development, potential use cases are emerging across industries and functions. A few examples include:'),
  Document(id='9d6012d1-6eb2-413b-80ca-a543c7fdc2d1', metadata={'source': 'https://www.adamsstreetpartners.com/insights/the-next-frontier-the-rise-of-agentic-ai/', 'title': 'The Next Frontier: The Rise of Agentic AI', 'description': 'Age

In [25]:
response['context']

[Document(id='06915818-815f-4150-b64d-2fddb9da0d94', metadata={'source': 'https://www.adamsstreetpartners.com/insights/the-next-frontier-the-rise-of-agentic-ai/', 'title': 'The Next Frontier: The Rise of Agentic AI', 'description': 'Agentic AI systems hold the potential to achieve specific goals by dynamically interacting with human and model inputs to make decisions autonomously.', 'language': 'en-US'}, page_content='What are the Potential Use Cases?\nWhile many agentic AI applications are in early development, potential use cases are emerging across industries and functions. A few examples include:'),
 Document(id='9d6012d1-6eb2-413b-80ca-a543c7fdc2d1', metadata={'source': 'https://www.adamsstreetpartners.com/insights/the-next-frontier-the-rise-of-agentic-ai/', 'title': 'The Next Frontier: The Rise of Agentic AI', 'description': 'Agentic AI systems hold the potential to achieve specific goals by dynamically interacting with human and model inputs to make decisions autonomously.', 'la